In [ ]:
import cv2
import numpy as np
import os
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
import keras.backend.tensorflow_backend as tfback
def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings)

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]
tfback._get_available_gpus = _get_available_gpus
tfback._get_available_gpus()
tf.config.list_logical_devices()
import keras
from keras import backend
from keras.models import Model
from keras.applications.vgg16 import VGG16
from scipy.optimize import fmin_l_bfgs_b
keras.backend.clear_session()

In [ ]:
def get_mask_spots(path , lower_range , higher_range):
    img = cv2.imread(path)
    img = cv2.resize(img , (768,512))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_range, higher_range)
    mask  = cv2.morphologyEx(cv2.medianBlur(mask , 3) , cv2.MORPH_CLOSE, np.ones((3,3) , np.uint8))
    res = cv2.bitwise_and(img , img , mask = mask)
    return mask , res

def add_spots(path , mask , spots):
    img = cv2.imread(path)
    img = cv2.resize(img , (768,512))
    mask_inv = cv2.bitwise_not(mask)
    img_bg = cv2.bitwise_and(img , img , mask = mask_inv)
    img_new = cv2.add(img_bg , spots)
    return img_new

In [ ]:
def randomize_mask(mask , spots):
    w = np.random.randint(mask.shape[1])
    h = np.random.randint(mask.shape[0])
    x = np.random.randint(mask.shape[1] - w)
    y = np.random.randint(mask.shape[0] - h)
    new_mask = new_mask[y:y+h , x:x+w]
    new_mask = cv2.resize(new_mask , (768,512))
    new_spots = new_spots[y:y+h , x:x+w]
    new_spots = cv2.resize(new_spots , (768,512))
    return new_mask , new_spots

In [ ]:
def nst(content , style):
    
    CHANNELS = 3
    IMAGE_SIZE = 500
    IMAGE_WIDTH = 768
    IMAGE_HEIGHT = 512
    IMAGENET_MEAN_RGB_VALUES = [123.68, 116.779, 103.939]
    CONTENT_WEIGHT = 0.02
    STYLE_WEIGHT = 4.5
    TOTAL_VARIATION_WEIGHT = 0.995
    TOTAL_VARIATION_LOSS_FACTOR = 1.25
    
    def content_loss(content, combination):
        return backend.sum(backend.square(combination - content))

    def gram_matrix(x):
        features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
        gram = backend.dot(features, backend.transpose(features))
        return gram

    def compute_style_loss(style, combination):
        style = gram_matrix(style)
        combination = gram_matrix(combination)
        size = IMAGE_HEIGHT * IMAGE_WIDTH
        return backend.sum(backend.square(style - combination)) / (4. * (CHANNELS ** 2) * (size ** 2))

    def total_variation_loss(x):
        a = backend.square(x[:, :IMAGE_HEIGHT-1, :IMAGE_WIDTH-1, :] - x[:, 1:, :IMAGE_WIDTH-1, :])
        b = backend.square(x[:, :IMAGE_HEIGHT-1, :IMAGE_WIDTH-1, :] - x[:, :IMAGE_HEIGHT-1, 1:, :])
        return backend.sum(backend.pow(a + b, TOTAL_VARIATION_LOSS_FACTOR))

    def evaluate_loss_and_gradients(x):
        x = x.reshape((1, IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
        outs = backend.function([xcombination], outputs)([x])
        loss = outs[0]
        gradients = outs[1].flatten().astype("float64")
        return loss, gradients

    class Evaluator:

        def loss(self, x):
            loss, gradients = evaluate_loss_and_gradients(x)
            self._gradients = gradients
            return loss

        def gradients(self, x):
            return self._gradients
    
    content_img = np.asarray(content, dtype="float32")
    content_img = np.expand_dims(content_img, axis=0)
    content_img[:, :, :, 0] -= IMAGENET_MEAN_RGB_VALUES[2]
    content_img[:, :, :, 1] -= IMAGENET_MEAN_RGB_VALUES[1]
    content_img[:, :, :, 2] -= IMAGENET_MEAN_RGB_VALUES[0]
    content_img = content_img[:, :, :, ::-1]

    style_image = np.asarray(style, dtype="float32")
    style_image = np.expand_dims(style_image, axis=0)
    style_image[:, :, :, 0] -= IMAGENET_MEAN_RGB_VALUES[2]
    style_image[:, :, :, 1] -= IMAGENET_MEAN_RGB_VALUES[1]
    style_image[:, :, :, 2] -= IMAGENET_MEAN_RGB_VALUES[0]
    style_image = style_image[:, :, :, ::-1]

    xcontent = backend.variable(content_img)
    xstyle = backend.variable(style_image)
    xcombination = backend.placeholder((1, IMAGE_HEIGHT, IMAGE_WIDTH, 3))

    input_tensor = backend.concatenate([xcontent,xstyle,xcombination], axis=0)
    model = VGG16(input_tensor=input_tensor, include_top=False)

    layers = dict([(layer.name, layer.output) for layer in model.layers])

    content_layer = 'block2_conv2'
    layer_features = layers[content_layer]

    content_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]

    loss = backend.variable(0.)
    loss = loss + CONTENT_WEIGHT * content_loss(content_image_features,
                                          combination_features)

    style_layers = ["block1_conv2", "block2_conv2", "block3_conv3", "block4_conv3", "block5_conv3"]
    for layer_name in style_layers:
        layer_features = layers[layer_name]
        style_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        style_loss = compute_style_loss(style_features, combination_features)
        loss += (STYLE_WEIGHT / len(style_layers)) * style_loss

    loss = loss + TOTAL_VARIATION_WEIGHT * total_variation_loss(xcombination)

    outputs = [loss]
    outputs += backend.gradients(loss, xcombination)

    evaluator = Evaluator()

    x = np.random.uniform(0, 255, (1, IMAGE_HEIGHT, IMAGE_WIDTH, 3)) - 128.

    for i in range(10):
        x, loss, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.gradients, maxfun=20)
        #print("Iteration %d completed with loss %d" % (i, loss))

    x = x.reshape((IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
    x = x[:, :, ::-1]
    x[:, :, 0] += IMAGENET_MEAN_RGB_VALUES[2]
    x[:, :, 1] += IMAGENET_MEAN_RGB_VALUES[1]
    x[:, :, 2] += IMAGENET_MEAN_RGB_VALUES[0]
    x = np.clip(x, 0, 255).astype("uint8")
    
    return x

In [ ]:
def generate(no_ofMasks=None):
    count = 486
    clean_paths = os.listdir('Clean/')
    damaged_paths = os.listdir('Damaged/')
    textures_paths = os.listdir('Textures/')
    if no_ofMasks is None:
        no_ofMasks = len(damaged_paths)
    no_ofTextures = len(textures_paths)
    for path_c in clean_paths[18:]:
        clean_path = 'Clean/' + path_c
        rng = np.random.default_rng()
        random_masks = rng.choice(len(damaged_paths) , size = no_ofMasks, replace=False)
        rng = np.random.default_rng()
        random_textures = None
        if no_ofMasks<=no_ofTextures:
            random_textures = rng.choice(no_ofTextures , size = no_ofMasks, replace=False)
        else:
            random_textures = []
            c = no_ofMasks
            while c>0:
                b = rng.choice(no_ofTextures , size = min(no_ofTextures,c), replace=False)
                c -= no_ofTextures
                random_textures += list(b)
        for n1,i in enumerate(random_masks):
            damaged_path = damaged_paths[i]
            values = np.split(np.array(damaged_path[:-4].split('_'),np.int32) , 2)
            mask , spots = get_mask_spots('Damaged/'+ damaged_path , values[0] , values[1])
            if n1>int(len(random_masks)/2-1):
                mask , spots = randomize_mask(mask , spots)
            damaged1 = add_spots(clean_path , mask , spots)
            style_path = 'Textures/' + textures_paths[random_textures[n1]]
            style = cv2.imread(style_path)
            style = cv2.resize(style , (768,512))
            print(count+1)
            damaged2 = nst(damaged1 , style)
            print(count+1 , "DONE")
            count += 1
            cv2.imwrite('Dataset/clean/'+str(count)+'.jpg' , cv2.resize(cv2.imread(clean_path) , (768,512)))
            cv2.imwrite('Dataset/damaged/'+str(count)+'.jpg' , damaged2)
            cv2.imwrite('Dataset/masks/'+str(count)+'.jpg' , mask)

In [ ]:
generate()